# 02 — Data Cleaning & Feature Engineering

本 Notebook 負責將原始 Titanic 資料轉換成適合模型使用的乾淨數據，確保後續建模品質穩定。

---

## 📌 目標

- 修正缺失值
- 製作新特徵（FamilySize）
- 將類別資料轉換成模型可讀格式
- 建立乾淨版 train dataset 作為 modeling input

---

## 📘 目錄

### 1. 匯入資料
- 讀取 train.csv
- 檢查資料格式與 columns

### 2. 處理缺失值
- Age → 以中位數補值
- Fare → 以中位數補值
- Embarked → 以眾數補值
- Cabin → 缺失過高，刪除整欄

### 3. 製作 FamilySize 特徵
- FamilySize = SibSp + Parch + 1
- 評估不同 FamilySize 的存活率（可選）

### 4. 類別變數編碼（Encoding）
- Sex → male=0, female=1
- Embarked → One-hot encoding（Embarked_Q, Embarked_S）

### 5. 輸出乾淨資料
- 顯示 head()
- 保存成乾淨版 train

---

## 🎯 最終成果

一份完整、可供模型使用的 **cleaned_train DataFrame**，  
確保模型不會受到缺失值或未處理特徵影響。

In [2]:
import pandas as pd 
import matplotlib as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
train = pd.read_csv('train.csv')
print(train.head())
train.info()

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
<c

In [3]:
#  建立 FamilySize（剛剛有做過，再做一次就好）
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1

#  補 Age（用中位數）
train["Age"] = train["Age"].fillna(train["Age"].median())

train["Age"] = train["Age"].astype(int)
#  刪掉 Cabin
train = train.drop(columns=["Cabin"])

#  補 Embarked（用眾數）
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])

#  Sex 映射成 0/1（這次是從原始 male/female 來）
train["Sex"] = train["Sex"].map({"male": 0, "female": 1})

#  Embarked 做 one-hot encoding
train = pd.get_dummies(train, columns=["Embarked"], drop_first=True)

In [ ]:
#  整體存活率
df = pd.read_csv("train.csv")
print(df.head())

print("\n=== 資訊概要 ===")
print(df.info())

print("\n=== 缺失值統計 ===")
print(df.isnull().sum())

# 目標欄位

target_col = "Survived"
survival_rate = df[target_col].mean()
print(f"\n整體存活率: {survival_rate:.3f} ({survival_rate*100:.2f}%)")

In [5]:
#  按 Sex 看存活率
sex_survival = df.groupby("Sex")[target_col].mean()
print("\n依性別的存活率：")
print(sex_survival)


依性別的存活率：
Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64


In [4]:
#  按 Pclass 看存活率
pclass_survival = df.groupby("Pclass")[target_col].mean()
print("\n依艙等的存活率：")
print(pclass_survival)


依艙等的存活率：
Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64


In [ ]:

#  年齡分組存活率（每 10 歲一組）
bins = range(0, 90, 10)
labels = [f"{i}-{i+9}" for i in bins[:-1]]
df["AgeGroup"] = pd.cut(df["Age"], bins=bins, labels=labels, right=True)

agegroup_survival = df.groupby("AgeGroup")[target_col].mean()
print("\n依年齡區間的存活率：")
print(agegroup_survival)

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,FamilySize,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",0,22,1,0,A/5 21171,7.2500,2,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38,1,0,PC 17599,71.2833,2,False,False
2,3,1,3,"Heikkinen, Miss. Laina",1,26,0,0,STON/O2. 3101282,7.9250,1,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,1,0,113803,53.1000,2,False,True
4,5,0,3,"Allen, Mr. William Henry",0,35,0,0,373450,8.0500,1,False,True


In [5]:
# 將清洗完的 train 存成一個新的檔案
train.to_csv("train_clean.csv", index=False)
print("已儲存清洗後資料為 train_clean.csv")

已儲存清洗後資料為 train_clean.csv
